In [1]:
import os
from pathlib import Path
from  keras.preprocessing import image
from PIL import Image
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
import numpy as np

Using TensorFlow backend.
/home/nitish/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nitish/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nitish/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nitish/.local/lib/python3.5/site-packages/tensorflow/python

In [2]:
!python -m pip install --user pillow

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
P=Path("./pokemon_images1/")
dirs=P.glob("*")
#print(dirs)
img_data=[]
labels=[]
#optional
image_paths=[]
#converting string labels to integers
label_dict={"pikachu":0,"bulbasor":1,"charmander":2}
label2pokemon={0:"pikachu",1:"bulbasor",2:"charmander"}
for folder_dir in dirs:
    #print(folder_dir)
    label=(str(folder_dir).split("/")[-1])
    
    
    cnt=0
    print(folder_dir)
    
    #iterate over folder directories 
    for img_path in folder_dir.glob("*.jpg"):
        #print(img_path)
        img=image.load_img(img_path,target_size=(40,40))
        img_array=image.img_to_array(img)    
        img_data.append(img_array)
        labels.append(label_dict[label])
        cnt+=1
    print(cnt)
    
    

In [4]:
import numpy as np
print(len(img_data))
print(len(labels))
print(labels)

0
0
[]


In [5]:
X=np.array(img_data)
Y=np.array(labels)
print(X.shape)
print(Y.shape)
#X=X.resize(229,40,40,3)
X=np.array(img_data)
print(type(X))
print(X.shape)

(0,)
(0,)
<class 'numpy.ndarray'>
(0,)


In [6]:
##### visualisation
def draw_img(img,label):
    plt.title(label2pokemon[label])
    plt.imshow(img)
    plt.axis("off")
    plt.show()
    
    
    

In [7]:
draw_img(X[2]/255.0,Y[2])

IndexError: index 2 is out of bounds for axis 0 with size 0

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from keras import backend as k
from sklearn.utils import shuffle
X,Y=shuffle(X,Y,random_state=2)
X=X/255.0


In [ ]:
print(Y)

In [ ]:
for i in range(1,20):
    draw_img(X[i].reshape(40,40,3),Y[i])

In [ ]:
split=int(X.shape[0]*0.8)
X_=np.array(X)
Y_=np.array(Y)
X=X_[:split,:]
Y=Y_[:split]
X_test=X_[split:,:]
Y_test=Y_[split:]



In [ ]:
X=X.reshape(X.shape[0],-1)
X_test=X_test.reshape(X_test.shape[0],-1)
print(X.shape)
print(X_test.shape)
print(type(Y))

In [ ]:
def one_hot(y,depth):
    m=y.shape[0]
    y_oht=np.zeros((m,depth))
    y_oht[np.arange(m),y]=1
    return y_oht
Y_=one_hot(Y,3)
Y_test_=one_hot(Y_test,3)
print(Y_.shape)
print(Y_test_.shape)

In [ ]:
def getModel():
    model = Sequential()
    model.add(Dense(64,activation='relu',input_shape=(4800,)))
    model.add(Dense(35,activation='relu'))
    model.add(Dense(3,activation='softmax'))
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
    return model

In [ ]:
model=getModel()
model.summary()


In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=10,baseline=3)

In [ ]:
hist=model.fit(X,Y_,epochs=15,verbose=1,validation_data=(X_test,Y_test_),batch_size=5,callbacks=[es])

In [ ]:
val_loss=hist.history['val_loss']
train_loss=hist.history['loss']

In [ ]:
import matplotlib.pyplot as plt
plt.plot(val_loss,label='val_loss')
plt.plot(train_loss,label='train_loss')
plt.legend()

In [ ]:
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']
plt.plot(train_acc,label='train_acc')
plt.plot(val_acc,label='val_acc')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
predictions=model.predict(X_test)
prediction=np.array((predictions))
labels=np.argmax(prediction,axis=-1)
print(labels)

In [ ]:
from sklearn.externals import joblib
joblib.dump(model,"pokemon_model.h5")
m=joblib.load("pokemon_model.h5")
